

*   Step1:Gaussian Modeling

*   Step2:Given a sequence of latencies, figure out the states(aka     character pairs)
*   Using n viterbi algorithm for computing the most likely character pair sequence
*   Inferring character sequence from character pairs
*   For the time being, generate 10 most likely sequences or n=10, decide later what best n should be after running signal images through CNN LSTM

Gaussian Modelling part: We have a dataset of mean and standard deviation of inter keystroke timing for each character pair. This dataset is a dictionary with character pair as keys and [sigma,mu] as vallues for each character pair

In [ ]:
#dataset: character_pair->(mu,sigma) 
import numpy as np
import scipy.stats
def gaussian_modeling(character_pair,time):
    return scipy.stats.norm(dataset[character_pair][0],datatset[character_pair][1]).pdf(time)

In [2]:
def prob(time,state):
    '''returns the value of p(state|time)'''
    sum = 0
    for elt in dataset:
        sum+=gaussian_modeling(elt,time)
    return gaussian_modeling(state,time)/sum


Steps of nviterbi algorithm:


*   Initialize the class with value of n and total states
*   When t=1, initialize all states with a value which is given the equal to P(state|time), store these values in a dictionary. Call this dictionary v. V[(q,1)]=p(q|y1) for all possible qs
*   The above value is got by the function above
*   Maintain a class variable of predesssor states(26) for each state[We have total 26*26 total states]
*   Defining variable for viterbi is n which is the total number of predecessor candidates for each state  
*   For time t>1, we have to find n best candidates for each state 
*   When t = 2, each state has total 26 ingoing arrows, we take the n best one among 26 states
*   This equation p[q_2|y_2]=p[y_2|q_2]p[q_2|q_1]p[q_1|y_1] for all predecessor q1 of q2 and select n best q1s for each state q2
*   Now start maintaining a dictionary with keys in the form (state,time) starting from time t=2 and like the above example (q2,2) will map to a list of (predecessor_state,value) so the n best predecessor states and the value of p[q2|y2] for each q1. Call that dictionary V. So V[(q2,2)]= [(q1,p[q2|y2]),(),(),()...]
*   Starting from t=3, for a state q, for all its predecessor q' and t-1, we look through the values of  (q',t-1) in the dictionary and write a function that finds the best ns among these 26*n values and then update the dictionary with these values-
V[(state,3)]=[(state1,state2,value),(state1,state2,value),(state1,state2,),()] value is 
p(q3|y3)=p(y3|q3)p(q3|q2)p(q2|y2)
*   At the given final state, we have 26*n paths, we take the n best among them and store them for future CNNLSTM purpose.
*   Finally, we save the n best candidates 


In [ ]:
def find_best_n(prev_list,n,new_state):
    if new state:
        result_list = []
        for i in range(len(prev_list)):
            if i<n:
                new_tup = (new_state,)+prev_list[i]
                result_list.append(new_tup)
            else:
                curr_min = min(l,key lambda e:e[-1])
                if curr_min<prev_list[i][-1]:
                    ind = result_list.index(curr_min)
                    result_list.pop(ind)
                    new_tup = (new_state,)+prev_list[i]
                    result_list.append(new_tup)
    else:
        result_list = []
        for i in range(len(prev_list)):
            if i<n:
                result_list.append(prev_list[i])
            else:
                curr_min = min(l,key lambda e:e[-1])
                if curr_min<prev_list[i][-1]:
                    ind = result_list.index(curr_min)
                    result_list.pop(ind)
                    result_list.append(prev_list[i])
    return result_list

In [4]:
#dataset: character_pair->(mu,sigma) 
def n_viterbi_algorithm(n,character_set,time_sequence):
    #Generate All states 
    states = set()
    for character1 in character_set:
        for character2 in character_set:
            states.add((character1,character2))
    predecessor_dictionary = {}
    for state in states:
        predecessor_dictionary[state]=[]
        for char in character_set:
            predecessor_dictionary[state].append((state[1],char))
    #generate values for time =1 and initialize a dictionary  
    V = {}
    for state in states:
        V[(state,1)] = [(state,prob(time_sequence[0],state))]
    for i in range(1,len(time_sequence)):
        for state in states:
            scale_value = gaussian_modeling(state,time_sequence[i])*(1/len(character_set))
            prev_list = []
            for prev in predecessor_dictionary[state]:
                prev_list.append(V[(prev,i)])
            modified_list = find_best_n(prev_list,n,state)
            for i in range(len(modified_list)):
                elt = modified_list[i][:-1]+(modified_list[i][-1]*scale_value,)
                modified_list[i] = elt
            V[(state,i+1)] = modified_list
    final_path_list = []
    for state in states:
        final_path_list.append(V[(state,len(time_sequence))])
    best_n_path = find_best_n(prev_list,n,new_state)
    return best_n_path